In [1]:
import os
import matplotlib.pyplot as plt
import RnnModel
import torch

from constants import model_params, base_data_path
from IPython.display import Image 
from matplotlib.pyplot import imshow
from utils import *


no display found. Using non-interactive Agg backend
No module named 'foundations'


In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# see issue #152
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# Directory from which we read the data
mode = "unlabeled"  # real, fake, or unlabeled

# Convert files to flac
# convert_to_flac(os.path.join(data_dir,mode))
dirpath = '../datasets/train/audios'
# filenames = os.listdir(dirpath)
# filenames = [f'{dirpath}/{filename}' for filename in filenames]
# filenames = filenames[:2]
filenames = ['032bf1a78f9280e7.aac']
print(filenames)

['032bf1a78f9280e7.aac']


In [3]:
print(dirpath)
# print(os.path.join(filenames[0], dirpath, mode))
processed_data = preprocess_from_filenames(
    filenames=filenames, dirpath=dirpath, mode='unlabeled',
    use_parallel=True
)

../datasets/train/audios


100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.55it/s]
/home/milselarch/projects/AISG/venv/lib/python3.9/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [4]:
print(model_params)
discriminator = RnnModel.Discriminator(
    num_freq_bin=model_params['num_freq_bin'],
    init_neurons=model_params['num_conv_filters'],
    num_conv_blocks=model_params['num_conv_blocks'],
    residual_con=model_params['residual_con'],
    num_dense_neurons=model_params['num_dense_neurons'],
    dense_dropout=model_params['dense_dropout'],
    num_dense_layers=model_params['num_dense_layers'],
)

{'num_freq_bin': 240, 'num_conv_blocks': 8, 'num_conv_filters': 32, 'spatial_dropout_fraction': 0.05, 'num_dense_layers': 1, 'num_dense_neurons': 50, 'dense_dropout': 0, 'learning_rate': 0.0001, 'epochs': 1, 'batch_size': 156, 'residual_con': 2, 'use_default': True, 'model_save_dir': 'fitted_objects'}


In [10]:
batch_x = processed_data[0]
batch_x = batch_x.reshape((
    len(batch_x), -1, hparams.num_mels
))

torch_batch = batch_x.transpose(0, 2, 1)
torch_batch = torch.from_numpy(torch_batch)
print(torch_batch.shape)

torch_batch = torch.vstack([
    torch_batch.unsqueeze(0), torch_batch.unsqueeze(0)
])

print(torch_batch.shape)


torch.Size([5095, 240, 1])
torch.Size([2, 5095, 240, 1])


In [6]:
torch.vstack([
    torch.ones((100, 1)).unsqueeze(0), 
    torch.ones((100, 1)).unsqueeze(0)
]).shape

torch.Size([2, 100, 1])

In [12]:
preds, hidden_states = discriminator(torch_batch)
print(preds.shape)
print(preds)

torch.Size([2, 1])
tensor([[0.5328],
        [0.5328]], grad_fn=<SigmoidBackward>)


In [7]:
rnn_output, hidden_states = discriminator.rnn_test(torch_batch)
print(rnn_output.shape)
print(hidden_states.shape)
sliced = rnn_output[:, -1, :][:, -1, :]
print(sliced.shape)

SHAPES [torch.Size([1, 5095, 50])]
BATCH-O torch.Size([1, 5095, 50])
torch.Size([1, 5095, 50])
torch.Size([1, 1, 50])


IndexError: too many indices for tensor of dimension 2

In [ ]:
rnn_output.contiguous().view(1, -1, 1).shape